# Introduction to Chinese Text Analysis

``2020-12-2``

Written by Zhiyuan Chen

Department of Trade Economics, Remin Business School

_Reference_: Hoberg, G., and G. Phillips. (2016), Text-based network industries and endogenous product differentiation, Journal of Political Economy, 124, 1423–65.

## Contents

- [中文分词工具](#中文分词工具)
    - [Jieba分词](#jieba分词)
    - [Baidu分词](#百度分词)   
- [去除停用词](#去除停用词)
- [提取目标词](#提取目标词)
- [词向量相似度](#词向量相似度)
    - [构造词向量](#构造词向量)
    - [Cosine距离](#Cosine距离)

## Jieba 分词
Github link: https://github.com/fxsjy/jieba/blob/master/README.md

In [ ]:
# pip install jieba
import jieba
import jieba.posseg as pseg 

#### 简单分词

In [ ]:
# 对一句话分词
sent = "小明选修了经济学与商务研究方法"
seg = jieba.cut(sent)
print('/'.join(seg))

In [ ]:
# the following code shows error
#sent1 = "这是一门中国人民大学商学院研究生必修课"
#sents = [sent, sent1]
#segs = jieba.cut(sents)
#print('/'.join(segs))

In [ ]:
#obtain list
seglist = jieba.lcut(sent)
print(seglist)

In [ ]:
# add user dictionary
jieba.load_userdict("userdict.txt")
seglist1 = jieba.lcut(sent)
print(seglist1)

#### 词性分词
结巴分词精确模式词性[标签列表](https://blog.csdn.net/fuhanghang/article/details/88064007)


In [13]:
import jieba.posseg as pseg
segpos = pseg.cut(sent)
print([se for se in segpos])

[pair('小明', 'nr'), pair('选修', 'v'), pair('了', 'ul'), pair('经济学与商务', 'add'), pair('研究方法', 'add')]


In [16]:
# list output
segposlist = pseg.lcut(sent)
print(segposlist)

[pair('小明', 'nr'), pair('选修', 'v'), pair('了', 'ul'), pair('经济学与商务', 'add'), pair('研究方法', 'add')]


To extract the segmented list, you need to know how to work with collections. A good reference can be found at [Collections](https://nbviewer.jupyter.org/github/zhiyuanryanchen/quantecon-notebooks-datascience/blob/master/python_fundamentals/collections.ipynb)

In [18]:
segposlist[0]

pair('小明', 'nr')

In [19]:
#check "collections.ipynb" in python_fundamentals in python datascience
tuple(segposlist[0])

('小明', 'nr')

In [22]:
list(segposlist[0])[1]

'nr'

In [24]:
#extract certain tags of interest
extract_tag=[]
for tag in segposlist:
    if list(tag)[1]=='nr':
        extract_tag.append(list(tag)[0])
extract_tag

['小明']

#### Paddle 模式


分词标签列表：

| 标签 | 含义     | 标签 | 含义     | 标签 | 含义     | 标签 | 含义     |
| ---- | -------- | ---- | -------- | ---- | -------- | ---- | -------- |
| n    | 普通名词 | f    | 方位名词 | s    | 处所名词 | t    | 时间     |
| nr   | 人名     | ns   | 地名     | nt   | 机构名   | nw   | 作品名   |
| nz   | 其他专名 | v    | 普通动词 | vd   | 动副词   | vn   | 名动词   |
| a    | 形容词   | ad   | 副形词   | an   | 名形词   | d    | 副词     |
| m    | 数量词   | q    | 量词     | r    | 代词     | p    | 介词     |
| c    | 连词     | u    | 助词     | xc   | 其他虚词 | w    | 标点符号 |
| PER  | 人名     | LOC  | 地名     | ORG  | 机构名   | TIME | 时间     |

In [25]:
# paddle mode
jieba.enable_paddle()
padseglist =  pseg.lcut(sent, use_paddle=True)
print(padseglist)

Paddle enabled successfully......


[pair('小明', 'PER'), pair('选修', 'v'), pair('了', 'u'), pair('经济学', 'n'), pair('与', 'c'), pair('商务', 'n'), pair('研究', 'vn'), pair('方法', 'n')]


## 百度分词
Github: https://github.com/baidu/lac

In [26]:
#pip install lac
from LAC import LAC
lac = LAC(mode='seg')

In [27]:
seg_lac = lac.run(sent)
print(seg_lac)

['小明', '选修', '了', '经济学', '与', '商务', '研究', '方法']


In [28]:
#run for lists
sent1 = "这是一门中国人民大学商学院研究生必修课"
sents = [sent, sent1]
seg_lacs = lac.run(sents)
print(seg_lacs)

[['小明', '选修', '了', '经济学', '与', '商务', '研究', '方法'], ['这', '是', '一门', '中国人民大学商学院', '研究生', '必修课']]


In [29]:
#add user-defined dictionary
from LAC import LAC
lac = LAC()
lac.load_customization('userdict_lac.txt', sep=None)
custom_seg = lac.run(sent)
print(custom_seg)

[['小明', '选修', '了', '经济学与商务', '研究方法'], ['PER', 'v', 'u', 'title', 'add']]


#### 增量训练
用户可以使用自己的数据，进行增量训练，首先需要将数据转换为模型输入的格式，并且所有数据文件均为"UTF-8"编码

## 去除停用词
[中文停用词词库](https://github.com/goto456/stopwords)

### 使用jieba分词

In [30]:
#import jieba
# 读取停顿词库
stk = [word.strip() for word in open('stop_words.txt',"r", encoding='utf-8').readlines()]
user_stk = [word.strip() for word in open('user_stopwords.txt', "r", encoding='utf-8').readlines()]
#读取文本数据
textfile = open('text.txt', 'r', encoding='utf-8')
text = [line.strip() for line in textfile]
print(text)

['公司经营范围主要为生产、销售糖、食用酒精、颗粒粕、电子元器件，计算机的研制、生产、推广应用、软件开发，计算机软硬件及外围设备，计算机配件、信息产业、电子高新技术产业、制糖机械设备等。', '火腿及火腿系列产品、腌腊制品、酱腌菜、肉类罐头；调味品（火腿系列）；批零；定型包装食品（卫生许可证有效期至2012年7月24日）；农副产品批零；出口本企业自产的产品。（凡涉及前置许可证及专项审批的凭相关有效证件经营）。', '莫高系列葡萄酒类生产、批发；葡萄原料、脱毒苗木、种条的繁育、销售；化学药品、原料药及制剂、中药制剂（仅限分支机构生产经营）；啤酒原料、花卉林木、农作物种子、饲草的种植、加工，中草药种植，畜牧养殖及畜产品加工，农业科技开发、咨询服务、培训，农副产品（不含粮食批发）及加工机械的批发零售；自营和代理各类商品及技术的进出口业务（国家限定或禁止的除外）、进料加工和"三来一补"对口贸易、转口贸易。主要产品为大麦芽、葡萄及葡萄酒、甘草系列产品。']


In [32]:
#jieba.enable_paddle()
seg = jieba.cut(text[0]) #cut_all=True for full classification
print(' '.join(seg))

公司 经营范围 主要 为 生产 、 销售 糖 、 食用酒精 、 颗粒 粕 、 电子 元器件 ， 计算机 的 研制 、 生产 、 推广应用 、 软件开发 ， 计算机 软硬件 及 外围设备 ， 计算机 配件 、 信息产业 、 电子 高新技术 产业 、 制糖 机械设备 等 。


In [34]:
# stop_words
wordscut = jieba.cut(text[1])
words_clean = ""
for w in wordscut:
    if not w in stk:
        #print(w)
        words_clean+= w+""
print(words_clean)

火腿火腿系列产品腌腊制品酱腌菜肉类罐头调味品火腿系列批零定型包装食品卫生许可证有效期2012年月24日农副产品批零出口企业自产产品涉及前置许可证专项审批相关有效证件经营


In [35]:
# user-defined stop_words
wordscut = jieba.cut(text[1])
words_clean = ""
for w in wordscut:
    if not w in user_stk:
        #print(w)
        words_clean+=w+""
print(words_clean)

火腿火腿腌腊制品酱腌菜肉类罐头调味品火腿批零定型包装食品2012年月24日农副产品批零出口自产涉及前置专项审批相关经营


In [36]:
#combine them together
file = open('text_nostop.txt','w',encoding='utf-8')
for t in text:
    wordscut = jieba.cut(t)
    line = ""
    for w in wordscut:
        if not w in user_stk:
        #print(w)
            line+=w+""
    file.write(line+'\n')
file.close()

In [37]:
text_nstp = [line.strip() for line in open('text_nostop.txt', 'r', encoding='utf-8')]
print(text_nstp)

['糖食用酒精颗粒粕电子元器件计算机研制推广应用软件开发计算机软硬件外围设备计算机配件信息产业电子高新技术制糖机械设备', '火腿火腿腌腊制品酱腌菜肉类罐头调味品火腿批零定型包装食品2012年月24日农副产品批零出口自产涉及前置专项审批相关经营', '莫高葡萄酒类批发葡萄原料脱毒苗木种条繁育化学药品原料药制剂中药制剂仅限分支机构经营啤酒原料花卉林木农作物种子饲草种植中草药种植畜牧养殖畜产品农业科技开发咨询服务培训农副产品含粮食批发机械批发零售自营各类商品技术进出口业务大麦芽葡萄葡萄酒甘草']


### 使用百度分词

In [38]:
from LAC import LAC
file = open('text_nostoplac.txt','w',encoding='utf-8')
for t in text:
    wordscut = lac.run(t)[0]
    line = ""
    for w in wordscut:
        if not w in user_stk:
        #print(w)
            line+=w+""
    file.write(line+'\n')
file.close()

In [ ]:
text_nstplac = [line.strip() for line in open('text_nostoplac.txt', 'r', encoding='utf-8')]
print(text_nstplac)

## 提取目标词

In [39]:
#extract certain tags of interested products
pdt_list=[]
for i, f in enumerate(text_nstp):
    segtags = pseg.lcut(f)
    f_list = []
    for tag in segtags:
        if list(tag)[1]=='n' or list(tag)[1]=='nz':
           f_list.append(list(tag)[0]) 
    #f_list = list(set(f_list))  # use 'set' to exclude repeated strings
    pdt_list.append(f_list)
print(pdt_list)

[['食用酒精', '颗粒', '电子', '元器件', '计算机', '计算机', '外围设备', '计算机', '配件', '信息产业', '电子', '高新技术', '制糖', '机械设备'], ['火腿', '火腿', '腌腊', '制品', '肉类', '罐头', '调味品', '火腿', '食品', '农副产品', '产', '前置', '专项'], ['葡萄酒', '葡萄', '原料', '条', '化学药品', '原料药', '制剂', '药制剂', '分支机构', '啤酒', '原料', '花卉', '农作物', '种子', '饲草', '畜牧', '畜产品', '农业', '咨询服务', '农副产品', '粮食', '机械', '商品', '技术', '业务', '葡萄', '葡萄酒', '甘草']]


## Work Remaining to be Done

* 丰富停顿词表
* 去除重复词
* 构建product space与企业产品向量 $V_i$
* 计算企业之间产品相似度